In [19]:
# Import findspark and initialise. 
import findspark
findspark.init()

In [20]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [24]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

# Read the CSV data from S3 into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the first few rows of the DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [25]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [26]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# Calculate the average price for a four-bedroom house sold in each year
average_prices_by_year = spark.sql("""
    SELECT
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        year
    ORDER BY
        year
""")
average_prices_by_year.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [27]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("""
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3
        AND bathrooms = 3
    GROUP BY
        date_built
    ORDER BY
        date_built
""")

# Show the result
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [28]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("""
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
    GROUP BY
        date_built
    ORDER BY
        date_built
""")

# Show the result
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [29]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SQL query to calculate the average price of homes greater than or equal to $350,000
result = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        price >= 350000
    GROUP BY
        view
    ORDER BY
        view
""")

# Show the result
result.show()

# End timing the query and print the runtime
end_time = time.time()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.41785502433776855 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [34]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

# SQL query to calculate the average price of homes greater than or equal to $350,000
result = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        price >= 350000
    GROUP BY
        view
    ORDER BY
        view
""")

# Show the result
result.show()

# End timing the query and print the runtime
end_time = time.time()


print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.25251340866088867 seconds ---


In [37]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("partitioned_home_sales")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/C:/Users/joey5/Desktop/Module 22/Starter_Code/Starter_Code/partitioned_home_sales already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
# 11. Read the formatted parquet data.
# Read the partitioned Parquet data
partitioned_home_sales_df = spark.read.parquet("partitioned_home_sales")

# Show the DataFrame to verify the data
partitioned_home_sales_df.show()


In [ ]:
# 12. Create a temporary table for the parquet data.
# Create a temporary table for the Parquet data
partitioned_home_sales_df.createOrReplaceTempView("parquet_table")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

# SQL query to calculate the average price of homes using the Parquet DataFrame
result_parquet = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        parquet_table
    WHERE
        price >= 350000
    GROUP BY
        view
    ORDER BY
        view
""")

# Show the result
result_parquet.show()

# End timing the query and print the runtime
end_time = time.time()
runtime = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("The 'home_sales' temporary table is still cached.")
else:
    print("The 'home_sales' temporary table is not cached.")